## 🗺️ Applying Joe’s Strategy with Data

In this notebook highlights:
* Identify neighborhood shopping centers using OpenStreetMap.
* Use H3 hexagons to approximate 5-mile trade areas.
* Pull in Census data to find areas with **high educational attainment**.
* Visualize the trade areas and overlay **Trader Joe’s** locations.

# 📦 Requirements

Before running this notebook, make sure you have the following packages installed:

- `osmnx` — for pulling OpenStreetMap data
- `geopandas` — for working with geospatial data
- `census` — for accessing the US Census API
- `shapely` — for geometric operations
- `h3` — for Uber’s hexagonal spatial indexing
- `folium` — for interactive mapping

You need census tract files for all of the areas you are looking and store those in folder 'census_data'

Lastly - you need a census API that is stored in a file called census_api.txt


In [70]:
import sys
#!{sys.executable} -m pip install h3~=3.0
# !{sys.executable} -m pip install census


## Step 1: Find all of the shopping centers
Note - OpenStreetMap is far from perfect on this, and my code below is very rough - and can be refined

In [14]:
import osmnx as ox
import geopandas as gpd

def find_neighborhood_shopping_centers(place_name):
    """
    Find neighborhood shopping centers in a given place using OpenStreetMap data.
    Excludes large malls.

    Args:
        place_name (str): The name of the place to search in (e.g., 'Minneapolis, Minnesota, USA').

    Returns:
        geopandas.GeoDataFrame: A GeoDataFrame containing the shopping centers.
    """
    # Query for general retail areas, marketplaces, and shopping centres but exclude malls
    tags = {
        'shop': True,             # any shop tag
        'landuse': ['retail'],    # landuse=retail
        'amenity': ['marketplace']  # small local shopping clusters
    }

    gdf = ox.features_from_place(place_name, tags)
    
    # Filter out features explicitly tagged as 'mall'
    if 'shop' in gdf.columns:
        gdf = gdf[gdf['shop'] != 'mall']
    
    # Optional: Clean up the data by selecting relevant columns
    #gdf = gdf.reset_index()[['osmid', 'shop', 'name', 'geometry']]
    
    return gdf



In [15]:
# Example usage:

place = "Minneapolis, Minnesota, USA"
neighborhood_centers = find_neighborhood_shopping_centers(place)

# Save to file
neighborhood_centers.to_file("minneapolis_neighborhood_centers.geojson", driver="GeoJSON")
print(f"Found {len(neighborhood_centers)} neighborhood shopping centers in {place}. Data saved to 'minneapolis_neighborhood_centers.geojson'.")


Found 1427 neighborhood shopping centers in Minneapolis, Minnesota, USA. Data saved to 'minneapolis_neighborhood_centers.geojson'.


In [16]:
neighborhood_centers

geometry  \
element id                                                              
node    453887805                          POINT (-93.24739 44.98397)   
        462681098                          POINT (-93.24262 44.98824)   
        544277961                          POINT (-93.32008 44.94927)   
        550932448                          POINT (-93.28832 44.94917)   
        571153113                          POINT (-93.29605 44.95788)   
...                                                               ...   
way     1346894624  POLYGON ((-93.28394 44.98719, -93.28369 44.987...   
        1365118949  POLYGON ((-93.22992 45.00295, -93.23027 45.003...   
        1365118950  POLYGON ((-93.22916 45.00263, -93.22909 45.002...   
        1383817933  POLYGON ((-93.21082 44.91256, -93.21083 44.912...   
        1383817945  POLYGON ((-93.20546 44.9082, -93.20515 44.9082...   

                                        name  \
element id                                     
node    453887805                Mr. Santana   
        462681098          8th Street Market   
        544277961             Barnes & Noble   
        550932448                        NaN   
        571153113   Kowalski's Uptown Market   
...                                      ...   
way     1346894624               The Foundry   
        1365118949    Quarry Shopping Center   
        1365118950    Quarry Shopping Center   
        1383817933                       NaN   
        1383817945                       NaN   

                                                        opening_hours  \
element id                                                              
node    453887805                Su-Th 09:00-03:00, Fr,Sa 09:00-04:00   
        462681098   Mo-Th 09:00-23:00; Fr, Sa 09:00-00:00; Su 10:0...   
        544277961     Mo-Sa 09:00-21:00; Sa-Su 10:00-20:00; PH closed   
        550932448                                                 NaN   
        571153113                              06:00-24:00; PH closed   
...                                                               ...   
way     1346894624                                                NaN   
        1365118949                                                NaN   
        1365118950                                                NaN   
        1383817933                                                NaN   
        1383817945                                                NaN   

                           shop addr:housenumber addr:postcode  \
element id                                                       
node    453887805   convenience              NaN           NaN   
        462681098   convenience              630         55414   
        544277961         books             3216         55416   
        550932448        vacant             2922         55408   
        571153113   supermarket             2440         55405   
...                         ...              ...           ...   
way     1346894624    houseware              905           NaN   
        1365118949          NaN              NaN           NaN   
        1365118950          NaN              NaN           NaN   
        1383817933          NaN              NaN           NaN   
        1383817945          NaN              NaN           NaN   

                              addr:street              alt_name  \
element id                                                        
node    453887805                     NaN                   NaN   
        462681098   Southeast 8th Street   Eighth Street Market   
        544277961        Lake Street West                   NaN   
        550932448    Lyndale Avenue South                   NaN   
        571153113   Hennepin Avenue South                   NaN   
...                                   ...                   ...   
way     1346894624       North 5th Street                   NaN   
        1365118949                    NaN                   NaN   
       

## Step 2 - develop a list of h3 hexes around these center

In [42]:
import geopandas as gpd
import pandas as pd
import h3
import numpy as np
from shapely.geometry import Point, Polygon
from shapely.ops import unary_union

from census import Census

# 1️⃣ Load your shopping centers
shopping_centers = gpd.read_file("minneapolis_neighborhood_centers.geojson")
shopping_centers = shopping_centers.to_crs(epsg=4326)  # WGS84 for lat/lon

# 2️⃣ Define radius in meters (5 miles ≈ 8,046.72 meters)
radius_meters = 8046.72

# 3️⃣ Choose an H3 resolution (8 is a good start)
h3_resolution = 8

# 4️⃣ For each center, create a 5-mile buffer and get H3 hexagons
def get_hexagons(lat, lon, radius, resolution):
    # Approximate the radius in degrees (approx. 1 deg ≈ 111 km)
    radius_deg = radius / 1000.0 / 111.0
    point = Point(lon, lat)
    circle = point.buffer(radius_deg)
    bounds = circle.bounds

    # Generate hexagons covering the bounding box
    hexagons = set()
    min_lat, min_lon, max_lat, max_lon = bounds[1], bounds[0], bounds[3], bounds[2]
    lat_steps = np.linspace(min_lat, max_lat, 50)
    lon_steps = np.linspace(min_lon, max_lon, 50)

    for lat_step in lat_steps:
        for lon_step in lon_steps:
            hex_id = h3.geo_to_h3(lat_step, lon_step, resolution)
            hex_boundary = Polygon(h3.h3_to_geo_boundary(hex_id, geo_json=True))
            if circle.intersects(hex_boundary):
                hexagons.add(hex_id)
    return hexagons

# 5️⃣ Collect all hexagons
all_hexagons = set()
for idx, row in shopping_centers.iterrows():
    lat = row.geometry.centroid.y
    lon = row.geometry.centroid.x
    hexagons = get_hexagons(lat, lon, radius_meters, h3_resolution)
    all_hexagons.update(hexagons)

# 6️⃣ Convert hexagons to GeoDataFrame
hex_geoms = []
for hex_id in all_hexagons:
    boundary = Polygon(h3.h3_to_geo_boundary(hex_id, geo_json=True))
    hex_geoms.append({'hex_id': hex_id, 'geometry': boundary})

hex_gdf = gpd.GeoDataFrame(hex_geoms, crs='EPSG:4326')



## Step 3 - Find the census data for these hexes. 

In [30]:
with open('census_api.txt', 'r') as file: # You’ll need your free Census API key - store in that file
    API_KEY = file.read().strip()


# 7️⃣ Load census tract shapefile
census_tracts = gpd.read_file("./census_data/tl_2024_27_tract.shp")
census_tracts = census_tracts.to_crs(epsg=4326)

# 8️⃣ Query ACS data: population & educational attainment
c = Census(API_KEY)

state_fips = "27"    # Minnesota
county_fips = "053"  # Hennepin County

variables = [
    "B01003_001E",  # Total population
    "B15003_001E",  # Pop 25+
    "B15003_022E",  # Bachelor's degree
    "B15003_023E",  # Master's degree
    "B15003_024E",  # Professional degree
    "B15003_025E"   # Doctorate degree
]

acs_data = c.acs5.state_county_tract(
    variables,
    state_fips,
    county_fips,
    Census.ALL
)

acs_df = pd.DataFrame(acs_data)

# Rename columns
acs_df.rename(columns={
    "B01003_001E": "total_population",
    "B15003_001E": "pop_25_over",
    "B15003_022E": "bach_deg",
    "B15003_023E": "mast_deg",
    "B15003_024E": "prof_deg",
    "B15003_025E": "phd_deg"
}, inplace=True)

# Calculate bachelor's or higher
acs_df["bach_or_higher"] = (
    acs_df["bach_deg"].astype(int) +
    acs_df["mast_deg"].astype(int) +
    acs_df["prof_deg"].astype(int) +
    acs_df["phd_deg"].astype(int)
)

# Calculate percentage
acs_df["pct_bach_or_higher"] = (
    acs_df["bach_or_higher"] / acs_df["pop_25_over"].replace(0, pd.NA)
) * 100

# Create GEOID for joining
acs_df["GEOID"] = (
    acs_df["state"].str.zfill(2) +
    acs_df["county"].str.zfill(3) +
    acs_df["tract"].str.zfill(6)
)


## Step 4 - Merge hexes and pop data

In [31]:

# 9️⃣ Merge ACS data with census tracts
census_tracts["GEOID"] = census_tracts["GEOID"].astype(str)
census_tracts = census_tracts.merge(
    acs_df[["GEOID", "total_population", "pct_bach_or_higher"]],
    on="GEOID",
    how="left"
)

# 🔟 Spatial join: assign tract populations & education to H3 hexagons
joined = gpd.sjoin(census_tracts, hex_gdf, how='inner', predicate='intersects')

# 1️⃣1️⃣ Aggregate by hex_id
agg = joined.groupby('hex_id').agg({
    'total_population': 'sum',
    'pct_bach_or_higher': 'mean'  # Use mean or weighted mean for accuracy
}).reset_index()

# 📊 Show result
print(agg.head())

            hex_id  total_population pct_bach_or_higher
0  88262cd001fffff            9702.0          78.292546
1  88262cd003fffff            7561.0          78.769156
2  88262cd005fffff           18085.0          78.229733
3  88262cd007fffff            9411.0          80.801511
4  88262cd009fffff            5444.0          74.179487


In [44]:
# Make sure hex_gdf has the correct 'hex_id' column
hex_gdf_temp = hex_gdf.merge(agg, on='hex_id', how='left')

shopping_centers = shopping_centers.to_crs(epsg=4326)
hex_gdf_temp = hex_gdf_temp.to_crs(epsg=4326)

shopping_centers_with_hex = gpd.sjoin(
    shopping_centers,
    hex_gdf_temp[['hex_id', 'geometry', 'total_population', 'pct_bach_or_higher']],
    how='left',
    predicate='within'
)

shopping_centers_with_hex.head()

,element,id,name,opening_hours,shop,addr:housenumber,addr:postcode,addr:street,alt_name,source,...,start_date:building,toilets:access,source:feature,roof:shape,type,geometry,index_right,hex_id,total_population,pct_bach_or_higher
0,node,453887805,Mr. Santana,"Su-Th 09:00-03:00, Fr,Sa 09:00-04:00",convenience,None,None,None,None,None,...,None,None,None,None,None,POINT (-93.24739 44.98397),438.0,8827526f2dfffff,22276.0,58.089543
1,node,462681098,8th Street Market,"Mo-Th 09:00-23:00; Fr, Sa 09:00-00:00; Su 10:0...",convenience,630,55414,Southeast 8th Street,Eighth Street Market,local_knowledge,...,None,None,None,None,None,POINT (-93.24262 44.98824),260.0,88275268dbfffff,18974.0,62.638434
2,node,544277961,Barnes & Noble,Mo-Sa 09:00-21:00; Sa-Su 10:00-20:00; PH closed,books,3216,55416,Lake Street West,None,None,...,None,None,None,None,None,POINT (-93.32008 44.94927),397.0,88262cd243fffff,9774.0,78.316521
3,node,550932448,None,None,vacant,2922,55408,Lyndale Avenue South,None,None,...,None,None,None,None,None,POINT (-93.28832 44.94917),176.0,8827526d65fffff,22454.0,54.063667
4,node,571153113,Kowalski's Uptown Market,06:00-24:00; PH closed,supermarket,2440,55405,Hennepin Avenue South,None,None,...,None,None,None,None,None,POINT (-93.29605 44.95788),220.0,88275268b3fffff,21492.0,58.275572


## Final step - map it out. 

In [69]:
import folium

# List of actual Trader Joe's locations
trader_joes_locations = [
    {
        'name': "Trader Joe's - St Louis Park",
        'lat': 44.93434957376787,
        'lon': -93.33723966374527
    },
    {
        'name': "Trader Joe's - Minneapolis",
        'lat': 44.977115691817296,
        'lon': -93.25766985895447
    },
    {
        'name': "Trader Joe's - Bloomington",
        'lat': 44.86557047524667, 
        'lon': -93.33392017946046
    }
    
]

# Find only the centers in areas with bachelor's degrees or higher of 75%
filtered_centers = shopping_centers_with_hex[
    shopping_centers_with_hex['pct_bach_or_higher'] > 75
]



# Convert polygons to centroids
shopping_centers_points = filtered_centers.copy()
shopping_centers_points['geometry'] = shopping_centers_points.geometry.centroid

# Create map
m = folium.Map(location=[44.9778, -93.2650], zoom_start=11)

# Add hexagons colored by % bachelor's or higher
folium.Choropleth(
    geo_data=hex_gdf_temp.to_json(),
    name='Education (Bachelors+)',
    data=hex_gdf_temp,
    columns=['hex_id', 'pct_bach_or_higher'],
    key_on='feature.properties.hex_id',
    fill_color='YlGnBu',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='% Bachelors or Higher'
).add_to(m)

# Add shopping centers
for idx, row in shopping_centers_points.iterrows():
    folium.CircleMarker(
        location=[row.geometry.y, row.geometry.x],
        radius=5,
        popup=f"{row.get('name', 'Shopping Center')}",
        color='red',
        fill=True,
        fill_opacity=0.7
    ).add_to(m)

# Add layer control
folium.LayerControl().add_to(m)

# Add large circles for Trader Joe's locations
for loc in trader_joes_locations:
    folium.Circle(
        location=[loc['lat'], loc['lon']],
        radius=500,  # radius in meters (1 km)
        popup=loc['name'],
        color='blue',
        fill=True,
        fill_opacity=0.75
    ).add_to(m)


# Save or display
m.save("shopping_centers_map.html")
m  # If running inside a Jupyter Notebook

/var/folders/lz/nxk1r4d55_l375zkwfk24t480000gn/T/ipykernel_74599/679562997.py:32: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  shopping_centers_points['geometry'] = shopping_centers_points.geometry.centroid


## 📝 Conclusion

This is a **very rough version** of the analysis that Joe Coulombe and his team might have done to pick out new Trader Joe’s locations. Joe described in his book that he and his team combed through data to identify promising sites, and this notebook focuses on just **one of the main criteria** he considered: **educational attainment**.

What’s particularly interesting in the Minneapolis area is that the **existing Trader Joe’s stores aren’t squarely inside the highest-educated trade areas**—but rather they sit on the edges. This suggests that while education levels are important, they are just one piece of the puzzle.

🔍 **Next steps** could include:
- Layering in **income levels** to find areas where high educational attainment aligns with being **underpaid**—Trader Joe’s core customer.
- Exploring other key criteria—like **retiree populations**, proximity to universities/hospitals, or older housing stock.

Joe would never have stopped at the data. His final step was always to **drive the area himself**, looking for **older neighborhoods** where people might have **excess money but not necessarily spend it all**. He’d avoid areas with too many speedboats or RVs—these were signs of high incomes but also high spending, which wasn’t the sweet spot for his stores.
